In [0]:
%run ../Utils/DBInstanceUtils

In [0]:
dbi = DBInstanceUtils('treinamento_formula1')
dbi.set_dbfs_filepath(dbi,'mnt', 'formula1/prata')

df_dict = {}

for folder in dbutils.fs.ls(dbi.get_dbfs_filepath()):
    for file in dbutils.fs.ls(folder.path):
        path = f'{folder.path}{file.name.removesuffix("/")}'
        df = (spark.read.format('delta').load(path))
        df_dict[file.name.removesuffix("Trusted.delta/")] = df

drivers = df_dict['drivers']
results = df_dict['results']

first_positioned = (
        drivers.join(results.filter(results.position.isNotNull() & (results.points > 1)),
                     drivers.driver_id == results.driver_id, 'inner')
        ).select(drivers.driver_id, 'name', 'nationality', 'position', 'points').display()